In [1]:
from pyspark import SparkConf, SparkContext
import os
conf = SparkConf().setMaster('local').setAppName('spark_app')
spark = SparkContext(conf = conf).getOrCreate()
spark

24/12/05 10:24:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local appName=spark_app>

In [2]:
#directory = ''
directory = os.path.join(os.getcwd(), 'data')
file_name = 'restaurant_reviews.csv'
file_path = os.path.join(directory, file_name)

In [3]:
lines = spark.textFile('file:///' + file_path.replace('\\','/'))
lines.take(3)

['id,item,cateogry,reviews,', '0,짜장면,중식,125,', '1,짬뽕,중식,235,']

In [6]:
header = lines.first()
header

'id,item,cateogry,reviews,'

In [9]:
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.take(3)

['0,짜장면,중식,125,', '1,짬뽕,중식,235,', '2,김밥,분식,32,']

In [14]:
def parse(row):
    field = row.split(',')
    category = field[2]
    reviews = int(field[3])

    return category, reviews

In [17]:
res1 = filtered_lines.map(parse).collect()
res2 = filtered_lines.map(parse).take(5)

In [18]:
res1, res2

([('중식', 125),
  ('중식', 235),
  ('분식', 32),
  ('분식', 534),
  ('일식', 223),
  ('일식', 52),
  ('일식', 12),
  ('아시안', 312),
  ('패스트푸드', 12),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23)],
 [('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)])

In [24]:
cat_reviews = filtered_lines.map(parse).persist() #persist를 통해 매번 캐시에 올리지 않음
cat_reviews

PythonRDD[19] at RDD at PythonRDD.scala:53

In [25]:
res1 = cat_reviews.collect()
res2 = cat_reviews.take(10)

In [27]:
res1, res2

([('중식', 125),
  ('중식', 235),
  ('분식', 32),
  ('분식', 534),
  ('일식', 223),
  ('일식', 52),
  ('일식', 12),
  ('아시안', 312),
  ('패스트푸드', 12),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23),
  ('패스트푸드', 23)],
 [('중식', 125),
  ('중식', 235),
  ('분식', 32),
  ('분식', 534),
  ('일식', 223),
  ('일식', 52),
  ('일식', 12),
  ('아시안', 312),
  ('패스트푸드', 12),
  ('패스트푸드', 23)])

In [28]:
spark.stop()

A와 B의 차이가 뭐니?

[A]
res1 = filtered_lines.map(parse)
res1.take(3)

[B]
res1 = filtered_lines.map(parse).persist()
res1.take(3)
ChatGPT의 말:
ChatGPT
Spark에서 [A]와 [B]의 차이는 **RDD의 캐싱(메모리에 저장)**과 관련이 있습니다. 이를 상세히 설명하겠습니다.

[A] res1 = filtered_lines.map(parse); res1.take(3)
이 코드는 RDD 연산을 실행할 때마다 계산을 반복합니다.
res1은 map(parse) 트랜스포메이션에 의해 생성된 PipelinedRDD입니다.
res1.take(3)를 호출하면:
filtered_lines의 모든 데이터를 대상으로 map(parse) 연산이 수행됩니다.
연산이 완료되면 결과 중 상위 3개의 값을 반환합니다.
트랜스포메이션 재실행:
res1.take(3)를 호출할 때마다 map(parse) 연산이 다시 실행됩니다. 즉, 동일한 데이터를 반복적으로 계산합니다.
성능 저하가 발생할 수 있습니다.
[B] res1 = filtered_lines.map(parse).persist(); res1.take(3)
이 코드는 persist() 메서드를 사용해 RDD를 메모리 또는 디스크에 저장합니다.
persist()를 호출하면:
res1의 첫 번째 액션(take, collect, count 등)이 실행될 때, Spark는 데이터를 계산하고 RDD의 중간 결과를 캐싱합니다.
이후 동일한 RDD(res1)에 대해 액션을 호출하면 캐싱된 데이터를 재사용합니다.
트랜스포메이션 실행 횟수 감소:
한 번 계산된 데이터를 재사용하므로, 동일한 데이터에 대한 반복적인 트랜스포메이션 수행이 방지됩니다.
성능 향상을 제공합니다.
주요 차이점 요약
특징	[A] 기본 연산	[B] persist() 사용
RDD 재계산	매번 트랜스포메이션이 재계산됩니다.	첫 번째 액션 이후, 캐싱된 데이터를 재사용합니다.
성능	반복적으로 데이터를 읽고 처리하므로 느릴 수 있습니다.	캐싱된 데이터를 사용하므로 반복 작업이 빠릅니다.
메모리 사용	추가적인 메모리나 디스크 자원이 사용되지 않습니다.	메모리나 디스크에 데이터가 저장됩니다.
데이터 재사용	동일한 데이터를 반복적으로 계산합니다.	동일한 데이터를 효율적으로 재사용합니다.
언제 [A]와 [B]를 사용할까?
[A] 사용:
RDD가 단일 연산으로 한 번만 사용되고, 결과를 재사용할 필요가 없을 때.
[B] 사용:
동일한 RDD에 대해 여러 번 액션을 호출하거나 결과를 반복적으로 사용할 때.
대량 데이터 처리에서 성능 최적화가 필요할 때.
persist()와 cache()의 관계
persist()는 데이터 저장 방식을 명시적으로 지정할 수 있습니다(예: 메모리, 디스크, 또는 혼합).
cache()는 persist(StorageLevel.MEMORY_AND_DISK)의 별칭으로, 데이터를 메모리에 저장하며, 메모리가 부족하면 디스크에 저장합니다.
예제 비교
[A]:
python
코드 복사
res1 = filtered_lines.map(parse)
print(res1.take(3))  # 첫 번째 트랜스포메이션 실행
print(res1.take(3))  # 다시 트랜스포메이션 실행 (중복 계산 발생)
[B]:
python
코드 복사
res1 = filtered_lines.map(parse).persist()
print(res1.take(3))  # 첫 번째 트랜스포메이션 실행 및 데이터 캐싱
print(res1.take(3))  # 캐싱된 데이터 사용 (중복 계산 없음)
성능 차이:
[A]는 두 번의 take 호출로 동일한 데이터를 두 번 계산.
[B]는 첫 번째 호출에서 데이터를 캐싱하고 두 번째 호출은 캐싱된 데이터를 사용.
결론
[A]는 메모리를 절약하지만, 반복적인 계산이 필요한 경우 성능이 저하됩니다.
[B]는 메모리를 더 사용하지만, 반복 작업에서 성능이 향상됩니다.
대량 데이터 처리와 반복 작업이 필요한 상황에서는 [B]를 추천합니다.